In [ ]:
import rasterio
from rasterio.windows import from_bounds, Window, get_data_window
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np

from rasterio.transform import Affine
from src.data.gdal_triplet_dataset_generator import GdalTripletDatasetGenerator

In [ ]:
generator = GdalTripletDatasetGenerator("C:\\Users\\Szymon\\projects\\aoiw\\datasets\\orto2020.vrt")
generator.generate_dataset("C:\\Users\\Szymon\\projects\\aoiw\\datasets\\orto2020split", "C:\\Users\\Szymon\\projects\\aoiw\\datasets\\orto2020pos")

In [ ]:
import math
def plot_rasterio_image(array: np.ndarray):
    image_array = np.moveaxis(array, 0, -1)
    plt.imshow(image_array)
    plt.show()

def save_raster_file(path, dataset, window, image):
    kwargs = dataset.meta.copy()
    kwargs.update({
        'driver': 'GTiff',
        'height': window.height,
        'width': window.width,
        'transform': rasterio.windows.transform(window, dataset.transform)})

    with rasterio.open(path, 'w', **kwargs) as dst:
        dst.write(image)

def get_randomly_shifted_image(dataset, image, shift_min_percent = 0.2, shift_max_percent = 0.8):
    image_width, image_height = image.width, image.height

    min_shift_x = math.floor(shift_min_percent * image_width)
    max_shift_x = math.floor(shift_max_percent * image_width)

    min_shift_y = math.floor(shift_min_percent * image_height)
    max_shift_y = math.floor(shift_max_percent * image_height)

    x_min_bound = 0
    x_max_bound = dataset.width - image_width
    y_min_bound = 0
    y_max_bound = dataset.height - image_height


    image_x, image_y = image.xy(0, 0)
    dataset_row, dataset_col = dataset.index(image_x, image_y)

    def valid_lower_point(threshold, value):
        return max(threshold, value)
    
    def valid_upper_point(threshold, value):
        return min(threshold, value)

    def valid_lower_x(x):
        return valid_lower_point(x_min_bound, x)
    
    def valid_lower_y(y):
        return valid_lower_point(y_min_bound, y)
    
    def valid_upper_x(x):
        return valid_upper_point(x_max_bound, x)
    
    def valid_upper_y(y):
        return valid_upper_point(y_max_bound, y)


    blex1 = np.arange(valid_lower_x(dataset_col - max_shift_x), valid_lower_x(dataset_col - min_shift_x))
    blex2 = np.arange(valid_upper_x(dataset_col + min_shift_x), valid_upper_x(dataset_col + max_shift_x))

    bley1 = np.arange(valid_lower_y(dataset_row - max_shift_y), valid_lower_y(dataset_row - min_shift_y))
    bley2 = np.arange(valid_upper_y(dataset_row + min_shift_y), valid_upper_y(dataset_row + max_shift_y))
    x_range = np.concatenate([
        blex1, blex2
    ])

    y_range = np.concatenate([
        bley1, bley2
    ])

    new_x = np.random.choice(x_range)
    new_y = np.random.choice(y_range)

    dataset_window = Window(new_x, new_y, image_width, image_height)
    p = dataset.read(window=dataset_window)
    return dataset_window, p


for i in range(10):
    perc = 1
    while perc >= 0.001:
        window, image = get_randomly_shifted_image(dataset, patch)
        percent_zeros = np.sum(image == 0) / np.size(image)
        if percent_zeros >= 0.001:
            plot_rasterio_image(image)
        else:
            path = f"im{i}.tif"
            save_raster_file(path, dataset, window, image)
        perc = percent_zeros



In [ ]:
w=Window(5000, 5000, 400, 400)
d = dataset.read([1,2,3], window=w)
plot_rasterio_image(d)

In [ ]:
window = get_data_window(xd)
window

In [ ]:
win_transform = dataset.window_transform(w)
win_transform

In [ ]:
offset_x = np.random.random_integers(-320, 320)
offset_y = np.random.random_integers(-320, 320)
offset_x, offset_y


In [ ]:
dataset.bounds

In [ ]:
dataset.transform.

In [ ]:
patch.transform

In [ ]:
patch.bounds

In [ ]:
y = dataset.height * 2 // 3
x = dataset.width * 2 // 3
rst = dataset.read([1,2,3], window=Window(x, y, 400, 400))

In [ ]:
new_x, new_y = x+offset_x, y+offset_y
new_x,new_y

In [ ]:

anchor = np.moveaxis(rst, 0, -1)
plt.imshow(anchor)
plt.show()

In [ ]:
close = dataset.read([1,2,3], window=Window(new_x, new_y, 400, 400))

In [ ]:
pos = np.moveaxis(close, 0, -1)
plt.imshow(pos)
plt.show()

In [ ]:
row, col = dataset.index(362726.92,362330.49)

In [ ]:
size = 400
school = dataset.read([1,2,3], window=Window(col-(size // 2), row-(size // 2), size, size))

school

In [ ]:
s = np.moveaxis(school, 0, -1)
plt.imshow(s)
plt.show()

In [ ]:
dataset.bounds

In [ ]:
from PIL import Image
im = Image.fromarray(s)
im.save("your_file.jpeg")